In [1]:
import os 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder, LabelBinarizer
from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.pipeline import Pipeline, FeatureUnion

from sklearn.linear_model import LinearRegression, LogisticRegression, Lasso, Ridge, ElasticNet

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, GradientBoostingRegressor, AdaBoostRegressor

from sklearn.neighbors import KNeighborsRegressor

from sklearn.svm import SVR, SVC, LinearSVC

from sklearn.model_selection import GridSearchCV

from sklearn.metrics import mean_squared_error

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

from sklearn.naive_bayes import GaussianNB

from sklearn.kernel_ridge import KernelRidge

from sklearn.decomposition import KernelPCA, PCA


import tensorflow as tf
from tensorflow import keras

import datetime as dt

from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import classification_report

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping




In [2]:
class MLDataPreProcessing():
    def __init__(self, input_data_folder = 'C:/Users/Marius/Desktop/Daily data/US_largest_stocks_with_KG_data_Oxford/') : 
        self.input_data_folder = input_data_folder

    def data_preprocess(self, name = 'BAC', start_date = '2014-01-01') : 
        data = pd.read_csv(self.input_data_folder + name + '.csv', index_col = 0)
        data.index = pd.to_datetime(data.index, yearfirst = True)
        data = data.sort_index()
        data = data.loc[start_date:]
        data = data.where(~np.isnan(data['y']), np.nan).dropna(how='all')
        data = data.reset_index()
        if 'index' in data.columns : 
            data.rename(columns = {'index':'Date'}, inplace = True)
        else : 
            data.rename(columns = {'0':'Date'}, inplace = True)
        return data
    
    def data_preprocess_fill_missing_values(self, name = 'BAC', start_date = '2014-01-01'):
        data = self.data_preprocess(name, start_date)
        data = data.ffill().fillna(value=0)
        return data
    
    def convert_output_values_to_binary(self, name = 'BAC', output_columns = ['y'], start_date = '2014-01-01') : 
        data = self.data_preprocess_fill_missing_values(name, start_date)
        data[output_columns] = data[output_columns].where(data[output_columns] <=0, 1)
        data[output_columns] = data[output_columns].where(data[output_columns] == 1, 0)
        return data
    
    def full_preprocessed_input_output_dataset(self, name = 'BAC', output_columns = ['y'], start_date = '2014-01-01') : 
        #data = self.data_preprocess_fill_missing_values(name)
        data = self.convert_output_values_to_binary(name, output_columns, start_date)
        X_full = data.drop(labels = output_columns, axis=1)
        output_columns = output_columns + ['Date']    # ['Date', 'Ticker']
        y_full = data[output_columns]
        return X_full, y_full
    
    def model_update_frequency(self, frequency = 'year') : 
        '''
        frequency types = 1. year, 2. month, 3. day; year taken as default
        '''
        dates = pd.bdate_range(start='1980-01-01', end = dt.datetime.today())
        if frequency == 'day' : 
            periods = pd.to_datetime(dates)
        elif frequency == 'month':
            periods = [prev_day for day, prev_day in zip(dates, dates.shift(1)) if prev_day.month != day.month]
            periods = pd.to_datetime(periods)
        else : 
            periods = [prev_day for day, prev_day in zip(dates, dates.shift(1)) if prev_day.year != day.year]
            periods = pd.to_datetime(periods)
        return periods


if __name__ == "__main__":
    pre_proc = MLDataPreProcessing()
    #pre_proc.data_preprocess()
    pre_proc.full_preprocessed_input_output_dataset()
    #pre_proc.model_update_frequency()

    
    '''
    data = pd.read_csv(pre_proc.input_data_folder + 'AAPL' + '.csv', index_col = 0)
    data.index = pd.to_datetime(data.index, yearfirst = True)

    print(data.reset_index()['0'])
    '''
    


 


In [3]:
class DNNMLTrainingModel():
    def __init__(self):
        self.pre = MLDataPreProcessing()
        self.input_folder = 'C:/Users/Marius/Desktop/Daily data/US_largest_stocks_with_KG_data_Oxford/'
        self.output_folder = 'C:/Users/Marius/Desktop/Daily data/US_largest_stocks_with_KG_data_ML_models_output/'
        self.max_days_fwd_return_in_test_set_y = 25
        self.sequential_ml_model = keras.models.Sequential()

    def conv_1d_wavenet_cnn_model(self, X_train, y_train, X_valid, y_valid):
        self.sequential_ml_model = keras.models.Sequential()
        self.sequential_ml_model.add(keras.layers.InputLayer(input_shape=(None,21)))
        for rate in (1, 2, 4, 8) * 2:    # NOTE: Original model goes up to 8
            self.sequential_ml_model.add(keras.layers.Conv1D(filters=20, kernel_size=2, padding="causal",
                                                            activation="relu", dilation_rate=rate))
        self.sequential_ml_model.add(keras.layers.Conv1D(filters=1, kernel_size=1, activation="sigmoid"))        
    
        optimizer = keras.optimizers.Adam(learning_rate=0.01)
        self.sequential_ml_model.compile(optimizer=optimizer, 
                                        loss="binary_crossentropy", 
                                        metrics=["accuracy"]
                                        )
        self.sequential_ml_model.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))
         
    def rnn_gru_model(self, X_train, y_train, X_valid, y_valid):
        
        self.sequential_ml_model = keras.models.Sequential([
            keras.layers.GRU(42, return_sequences=True, kernel_initializer="he_normal",
                            input_shape=(None,21)),
            keras.layers.Dropout(0.5),
            keras.layers.GRU(42, kernel_initializer="he_normal", return_sequences=True),
            keras.layers.Dropout(0.5),
            keras.layers.TimeDistributed(keras.layers.Dense(1, activation='sigmoid'))
        ])
        
        optimizer = keras.optimizers.Adam(learning_rate=0.01)
        self.sequential_ml_model.compile(optimizer=optimizer, 
                                        loss="binary_crossentropy", 
                                        metrics=["accuracy"])
        self.sequential_ml_model.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))

    def ml_model_predictions(self, X_test, y_test_with_dates):
        y_pred = self.sequential_ml_model.predict(X_test)
        y_pred = y_pred.reshape(y_pred.shape[1], 1)
        y_pred_with_dates = pd.DataFrame(index = y_test_with_dates.index, data = y_pred)
        #y_pred_with_other_data = pd.concat([pd.DataFrame(y_test_with_dates.reset_index()), pd.DataFrame(y_pred)], axis=1)
        #y_pred_with_other_data = y_pred_with_other_data.set_index('Date')
        return y_pred_with_dates      
    
    
    def dnn_training_loop(self, ml_model_name = 'cnn_1d', dataset_name = 'BAC', output_columns_list = ['y'], model_update_frequency='year', start_date = '2014-01-01'):
        X_full, y_full = self.pre.full_preprocessed_input_output_dataset(name=dataset_name, output_columns=output_columns_list, start_date=start_date)
        
        update_frequency = self.pre.model_update_frequency(frequency=model_update_frequency)
        update_period = pd.to_datetime([date for date in update_frequency if date.year >= X_full['Date'].iloc[0].year])
        
        predictions_y_hat_all_periods = pd.DataFrame()
        for period_t, period_t_1 in zip(update_period[1:],update_period[2:]) : # If yearly, leave at least 1 year to train
            last_day_of_training_period_X = period_t - pd.offsets.BDay(self.max_days_fwd_return_in_test_set_y)
            #print(last_day_of_training_period_X, period_t, period_t_1)

            X_train_with_dates = X_full.set_index('Date').loc[:last_day_of_training_period_X]
            y_train_with_dates = y_full.set_index('Date').loc[:last_day_of_training_period_X]

            X_train_full = X_train_with_dates.reset_index().drop('Date', axis=1)
            y_train_full = y_train_with_dates.reset_index().drop('Date', axis=1)

            X_train, X_valid, y_train, y_valid = train_test_split(X_train_full.values, y_train_full.values, train_size=0.9, shuffle=False)

            #print(X_train.shape, y_train.shape)
            print(X_valid.shape, y_valid.shape)

            X_test_with_dates = X_full.set_index('Date').loc[period_t : period_t_1]
            y_test_with_dates = y_full.set_index('Date').loc[period_t : period_t_1]

            X_test = X_test_with_dates.reset_index().drop(['Date'], axis=1)
            y_test = y_test_with_dates.reset_index().drop(['Date'], axis=1)

            scaler = StandardScaler()
            X_train = scaler.fit_transform(X_train)
            X_valid = scaler.transform(X_valid)
            X_test = scaler.transform(X_test)

            X_train = X_train.reshape(1, X_train.shape[0], X_train.shape[1])
            X_valid = X_valid.reshape(1, X_valid.shape[0], X_valid.shape[1])
            X_test =  X_test.reshape(1,  X_test.shape[0],  X_test.shape[1])

            y_train = y_train.reshape(1, y_train.shape[0], y_train.shape[1])
            y_valid = y_valid.reshape(1, y_valid.shape[0], y_valid.shape[1])
            y_test =  y_test.values.reshape(1,  y_test.values.shape[0],  y_test.values.shape[1])    

            # Compile and run the Machine Learning model
            if ml_model_name == 'cnn_1d':
                self.conv_1d_wavenet_cnn_model(X_train, y_train, X_valid, y_valid)
            elif ml_model_name == 'rnn_gru':
                self.rnn_gru_model(X_train, y_train, X_valid, y_valid)
            else : 
                'ERROR: Specifiy the correct ML model'
                break

            # Make predictions for a given text batch
            batch_predictions = self.ml_model_predictions(X_test, y_test_with_dates)

            predictions_y_hat_all_periods = pd.concat([predictions_y_hat_all_periods, batch_predictions], axis=0)
            #predictions_y_hat_all_periods.to_csv(self.output_folder + dataset_name + '_predictions_test' + '.csv')

        predictions_y_hat_all_periods.rename(columns={0: dataset_name}, inplace=True)
        return predictions_y_hat_all_periods

    def dnn_training_loop_all_stocks(self, ml_model_name = 'rnn_gru', output_columns_list = ['y'], model_update_frequency='year', start_date = '2014-01-01') :
        predictions_all_stocks =pd.DataFrame()
        exceptions = []
        for idx, stock in enumerate(os.listdir(self.input_folder)) : 
            try : 
                print(stock)
                dataset_name = stock.split('.')[0]
                predictions_single_stock = self.dnn_training_loop(ml_model_name=ml_model_name, dataset_name=dataset_name, output_columns_list = output_columns_list,
                                                                model_update_frequency=model_update_frequency, start_date = start_date)
                predictions_all_stocks = pd.concat([predictions_all_stocks, predictions_single_stock], axis=1)
                predictions_all_stocks.to_csv(self.output_folder + 'US_largest_stocks_predictions_'+ ml_model_name + '.csv')
            except : 
                exceptions.append(stock)
                continue
        print(exceptions, 'exceptions')


if __name__ == "__main__":
    dnn_ml_model = DNNMLTrainingModel()
    #dnn_ml_model.dnn_training_loop()
    dnn_ml_model.dnn_training_loop_all_stocks()


AAPL.csv
(23, 21) (23, 1)


c:\Python39\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


Epoch 1/20
1/1 [==============================] - 7s 7s/step - loss: 0.8241 - accuracy: 0.4903 - val_loss: 0.5995 - val_accuracy: 0.6957
Epoch 2/20
1/1 [==============================] - 0s 199ms/step - loss: 0.7753 - accuracy: 0.5146 - val_loss: 0.6000 - val_accuracy: 0.6957
Epoch 3/20
1/1 [==============================] - 0s 170ms/step - loss: 0.7694 - accuracy: 0.5146 - val_loss: 0.6142 - val_accuracy: 0.6957
Epoch 4/20
1/1 [==============================] - 0s 164ms/step - loss: 0.7643 - accuracy: 0.5583 - val_loss: 0.6574 - val_accuracy: 0.6087
Epoch 5/20
1/1 [==============================] - 0s 169ms/step - loss: 0.7030 - accuracy: 0.5777 - val_loss: 0.6928 - val_accuracy: 0.4783
Epoch 6/20
1/1 [==============================] - 0s 169ms/step - loss: 0.7149 - accuracy: 0.6117 - val_loss: 0.7256 - val_accuracy: 0.2609
Epoch 7/20
1/1 [==============================] - 0s 169ms/step - loss: 0.6872 - accuracy: 0.5777 - val_loss: 0.7390 - val_accuracy: 0.2174
Epoch 8/20
1/1 [=======

c:\Python39\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


Epoch 1/20
1/1 [==============================] - 7s 7s/step - loss: 0.7926 - accuracy: 0.4954 - val_loss: 0.8702 - val_accuracy: 0.4898
Epoch 2/20
1/1 [==============================] - 0s 336ms/step - loss: 0.8324 - accuracy: 0.4676 - val_loss: 0.8259 - val_accuracy: 0.4898
Epoch 3/20
1/1 [==============================] - 0s 319ms/step - loss: 0.7211 - accuracy: 0.5440 - val_loss: 0.7592 - val_accuracy: 0.4490
Epoch 4/20
1/1 [==============================] - 0s 306ms/step - loss: 0.7343 - accuracy: 0.5162 - val_loss: 0.7349 - val_accuracy: 0.3878
Epoch 5/20
1/1 [==============================] - 0s 321ms/step - loss: 0.7436 - accuracy: 0.5370 - val_loss: 0.7235 - val_accuracy: 0.4082
Epoch 6/20
1/1 [==============================] - 1s 622ms/step - loss: 0.7161 - accuracy: 0.5625 - val_loss: 0.7155 - val_accuracy: 0.4286
Epoch 7/20
1/1 [==============================] - 1s 643ms/step - loss: 0.7098 - accuracy: 0.5579 - val_loss: 0.7097 - val_accuracy: 0.4898
Epoch 8/20
1/1 [=======

...

Epoch 1/20
1/1 [==============================] - 7s 7s/step - loss: 0.8186 - accuracy: 0.5072 - val_loss: 0.7275 - val_accuracy: 0.5568
Epoch 2/20
1/1 [==============================] - 2s 2s/step - loss: 0.7868 - accuracy: 0.5018 - val_loss: 0.7157 - val_accuracy: 0.5514
Epoch 3/20
1/1 [==============================] - 2s 2s/step - loss: 0.7713 - accuracy: 0.5006 - val_loss: 0.7233 - val_accuracy: 0.5459
Epoch 4/20
1/1 [==============================] - 2s 2s/step - loss: 0.7617 - accuracy: 0.5108 - val_loss: 0.7216 - val_accuracy: 0.5514
Epoch 5/20
1/1 [==============================] - 2s 2s/step - loss: 0.7370 - accuracy: 0.5253 - val_loss: 0.7081 - val_accuracy: 0.5297
Epoch 6/20
1/1 [==============================] - 2s 2s/step - loss: 0.7277 - accuracy: 0.5235 - val_loss: 0.7013 - val_accuracy: 0.5351
Epoch 7/20
1/1 [==============================] - 2s 2s/step - loss: 0.7038 - accuracy: 0.5343 - val_loss: 0.6996 - val_accuracy: 0.5514
Epoch 8/20
1/1 [=========================

c:\Python39\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


Epoch 1/20
1/1 [==============================] - 8s 8s/step - loss: 0.7881 - accuracy: 0.5008 - val_loss: 0.7342 - val_accuracy: 0.5286
Epoch 2/20
1/1 [==============================] - 2s 2s/step - loss: 0.7821 - accuracy: 0.5077 - val_loss: 0.7254 - val_accuracy: 0.4714
Epoch 3/20
1/1 [==============================] - 2s 2s/step - loss: 0.7812 - accuracy: 0.4881 - val_loss: 0.7241 - val_accuracy: 0.4524
Epoch 4/20
1/1 [==============================] - 2s 2s/step - loss: 0.7589 - accuracy: 0.5188 - val_loss: 0.7400 - val_accuracy: 0.5095
Epoch 5/20
1/1 [==============================] - 2s 2s/step - loss: 0.7445 - accuracy: 0.5225 - val_loss: 0.7435 - val_accuracy: 0.5095
Epoch 6/20
1/1 [==============================] - 2s 2s/step - loss: 0.7505 - accuracy: 0.5008 - val_loss: 0.7313 - val_accuracy: 0.4810
Epoch 7/20
1/1 [==============================] - 2s 2s/step - loss: 0.7177 - accuracy: 0.5379 - val_loss: 0.7276 - val_accuracy: 0.4762
Epoch 8/20
1/1 [=========================

c:\Python39\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


Epoch 1/20
1/1 [==============================] - 8s 8s/step - loss: 0.8143 - accuracy: 0.4993 - val_loss: 0.8531 - val_accuracy: 0.4255
Epoch 2/20
1/1 [==============================] - 3s 3s/step - loss: 0.8157 - accuracy: 0.4960 - val_loss: 0.7684 - val_accuracy: 0.3915
Epoch 3/20
1/1 [==============================] - 3s 3s/step - loss: 0.7890 - accuracy: 0.4827 - val_loss: 0.7209 - val_accuracy: 0.4468
Epoch 4/20
1/1 [==============================] - 3s 3s/step - loss: 0.7635 - accuracy: 0.5097 - val_loss: 0.7123 - val_accuracy: 0.5702
Epoch 5/20
1/1 [==============================] - 3s 3s/step - loss: 0.7644 - accuracy: 0.5069 - val_loss: 0.7100 - val_accuracy: 0.5489
Epoch 6/20
1/1 [==============================] - 3s 3s/step - loss: 0.7454 - accuracy: 0.5234 - val_loss: 0.7139 - val_accuracy: 0.4638
Epoch 7/20
1/1 [==============================] - 3s 3s/step - loss: 0.7321 - accuracy: 0.5372 - val_loss: 0.7310 - val_accuracy: 0.4085
Epoch 8/20
1/1 [=========================

In [4]:
# Classical ML models
class CNNMLTrainingModel():
    def __init__(self):
        self.pre = MLDataPreProcessing()
        self.input_folder = 'C:/Users/Marius/Desktop/Daily data/US_largest_stocks_with_KG_data_Oxford/'
        self.output_folder = 'C:/Users/Marius/Desktop/Daily data/US_largest_stocks_with_KG_data_ML_models_output/'
        self.max_days_fwd_return_in_test_set_y = 25        
        
        logModel = LogisticRegression(solver='liblinear')

    def logistic_regression_model_train(self, X_train, X_valid, y_train, y_valid):
        model = LogisticRegression(solver='liblinear')
        model.fit(X_train, y_train)
        model_score = model.score(X_valid, y_valid)

        print(f"Logistic regression Classifier Score = {model_score:.3f}")

        return model

    def logistic_regression_model_predict(self, X_train, X_valid, y_train, y_valid):
        model = self.logistic_regression_model_train(X_train, X_valid, y_train, y_valid)
        predictions_model = model.predict(X_valid)
        cm = confusion_matrix(y_valid, predictions_model, labels=model.classes_)
        disp = ConfusionMatrixDisplay(confusion_matrix=cm,
        display_labels=model.classes_)
        disp.plot()
        plt.show()
        print(cm)

        print('Logistic regression Regularized report table')
        print(classification_report(y_valid, predictions_model))


    def random_forest_train(self, X_train, X_valid, y_train, y_valid, max_depth, max_features):
        model = RandomForestClassifier(max_depth=max_depth, max_features = max_features)
        model.fit(X_train, y_train)
        model_score = model.score(X_valid, y_valid)

        print(f"Random Forest Classifier Score = {model_score:.3f}")

        return model


    def random_forest_predict(self, X_train, X_valid, y_train, y_valid, max_depth, max_features):
        model = self.random_forest_train(X_train, X_valid, y_train, y_valid, max_depth, max_features)
        predictions_model = model.predict(X_valid)
        cm = confusion_matrix(y_valid, predictions_model, labels=model.classes_)
        disp = ConfusionMatrixDisplay(confusion_matrix=cm,
        display_labels=model.classes_)
        disp.plot()
        plt.show()
        print(cm)

        print('Random Forest Classifier Regularized report table')
        print(classification_report(y_valid, predictions_model))


    def test(self, dataset_name = 'AAPL', output_columns_list = ['y'], model_update_frequency='year', start_date = '2014-01-01'):
        X_full, y_full = self.pre.full_preprocessed_input_output_dataset(name=dataset_name, output_columns=output_columns_list, start_date=start_date)

        X_full = X_full.drop('Date', axis=1)
        y_full = y_full.drop('Date', axis=1)

        proportion_of_positive_days_full = y_full.where(y_full == 1, np.nan).dropna().count() / y_full.count()

        X_train, X_valid, y_train, y_valid = train_test_split(X_full, y_full, train_size=0.90, shuffle=False)

        proportion_of_positive_days_valid = y_valid.where(y_full == 1, np.nan).dropna().count() / y_valid.count()
 
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_valid = scaler.transform(X_valid)
        #X_test = scaler.transform(X_test)

        print(proportion_of_positive_days_full, 'proportion_of_positive_days_full')
        print(proportion_of_positive_days_valid, 'proportion_of_positive_days_valid')

        #self.random_forest_train(X_train=X_train, X_valid=X_valid, y_train=y_train, y_valid=y_valid, max_depth = 10, max_features = 10)

        self.random_forest_predict(X_train=X_train, X_valid=X_valid, y_train=y_train, y_valid=y_valid, max_depth = 8, max_features = 8)

        #self.logistic_regression_model_train(X_train=X_train, X_valid=X_valid, y_train=y_train, y_valid=y_valid)

        #self.logistic_regression_model_predict(X_train=X_train, X_valid=X_valid, y_train=y_train, y_valid=y_valid)

    def cml_training_loop(self, ml_model_name = 'logistic_regression', dataset_name = 'MSFT', output_columns_list = ['y'], model_update_frequency='year', start_date = '2014-01-01') : 
        X_full, y_full = self.pre.full_preprocessed_input_output_dataset(name=dataset_name, output_columns=output_columns_list, start_date=start_date)

        update_frequency = self.pre.model_update_frequency(frequency=model_update_frequency)
        update_period = pd.to_datetime([date for date in update_frequency if date.year >= X_full['Date'].iloc[0].year])
        
        predictions_y_hat_all_periods = pd.DataFrame()
        for period_t, period_t_1 in zip(update_period[1:],update_period[2:]) : # If yearly, leave at least 1 year to train
            last_day_of_training_period_X = period_t - pd.offsets.BDay(self.max_days_fwd_return_in_test_set_y)
            #print(last_day_of_training_period_X, period_t, period_t_1)

            X_train_with_dates = X_full.set_index('Date').loc[:last_day_of_training_period_X]
            y_train_with_dates = y_full.set_index('Date').loc[:last_day_of_training_period_X]

            X_train_full = X_train_with_dates.reset_index().drop('Date', axis=1)
            y_train_full = y_train_with_dates.reset_index().drop('Date', axis=1)

            #X_train, X_valid, y_train, y_valid = train_test_split(X_train_full.values, y_train_full.values, train_size=0.9, shuffle=False)

            X_test_with_dates = X_full.set_index('Date').loc[period_t : period_t_1]
            y_test_with_dates = y_full.set_index('Date').loc[period_t : period_t_1]

            X_test = X_test_with_dates.reset_index().drop(['Date'], axis=1)
            y_test = y_test_with_dates.reset_index().drop(['Date'], axis=1)

            scaler = StandardScaler()
            X_train_full = scaler.fit_transform(X_train_full)
            #X_valid = scaler.transform(X_valid)
            X_test = scaler.transform(X_test)

            # Train selected CML model
            if ml_model_name == 'logistic_regression': 
                model = LogisticRegression(C = 0.1)
            elif ml_model_name == 'random_forest':
                model = RandomForestClassifier(n_estimators=1000, max_depth= 1, max_features = 1)
            else : 
                'ERROR: Specifiy the correct ML model'
                break
            model.fit(X_train_full, y_train_full.values.flatten())

            # Initialize and train logistic regression model

            # Check the classes_ attribute
            print("Class labels:", model.classes_)

            # Make test set predictions
            y_pred = model.predict_proba(X_test)

            # Probability of the predicted output being 1 i.e. positive daily return
            y_pred = y_pred[:,1]
            #print(y_pred)
            #y_pred = y_pred.reshape(y_pred.shape[1], 1)
            y_pred_with_dates = pd.DataFrame(index = y_test_with_dates.index, data = y_pred)
            #print(y_pred_with_dates)

            #y_pred_with_dates.rename(columns={0: dataset_name }, inplace=True)

            # Predictions added to other predictions' dataframe
            predictions_y_hat_all_periods = pd.concat([predictions_y_hat_all_periods, y_pred_with_dates], axis=0)
            

            proportion_of_positive_days_test_set = y_test.where(y_full == 1, np.nan).dropna().count() / y_test.count()
            
            #print(proportion_of_positive_days_test_set, 'proportion_of_positive_days_test_set')

            model_score = model.score(X_test, y_test)

            #print(f"Random Forest Classifier Score = {model_score:.3f}")
        predictions_y_hat_all_periods.rename(columns={0: dataset_name}, inplace=True)
        #print(predictions_y_hat_all_periods)
        #predictions_y_hat_all_periods.to_csv(self.output_folder + dataset_name + '_predictions_logistic_regression_test' + '.csv')
        return predictions_y_hat_all_periods

    def cml_training_loop_all_stocks(self,ml_model_name = 'random_forest', output_columns_list = ['y'], model_update_frequency='year', start_date = '2014-01-01') :    # 'random_forest', 'logistic_regression'
        predictions_all_stocks =pd.DataFrame()
        exceptions = []
        for idx, stock in enumerate(os.listdir(self.input_folder)) : 
            try : 
                print(stock)
                dataset_name = stock.split('.')[0]
                predictions_single_stock = self.cml_training_loop(ml_model_name=ml_model_name, dataset_name=dataset_name, output_columns_list = output_columns_list,
                                                                model_update_frequency=model_update_frequency, start_date = start_date)
                predictions_all_stocks = pd.concat([predictions_all_stocks, predictions_single_stock], axis=1)
                #predictions_all_stocks.to_csv(self.output_folder + 'US_largest_stocks_predictions_'+ ml_model_name + '.csv')
            except : 
                exceptions.append(stock)
                continue
        print(exceptions, 'exceptions')


if __name__ == "__main__":
    cml_ml_model = CNNMLTrainingModel()
    #cml_ml_model.test()

    #cml_ml_model.cml_training_loop()
    cml_ml_model.cml_training_loop_all_stocks()

AAPL.csv
Class labels: [0. 1.]
[[0.42587422 0.57412578]
 [0.42567789 0.57432211]
 [0.40249513 0.59750487]
 [0.40444063 0.59555937]
 [0.40816603 0.59183397]
 [0.43384752 0.56615248]
 [0.43042928 0.56957072]
 [0.40930069 0.59069931]
 [0.41166415 0.58833585]
 [0.42128951 0.57871049]
 [0.40816538 0.59183462]
 [0.41849417 0.58150583]
 [0.42700839 0.57299161]
 [0.42602271 0.57397729]
 [0.42707657 0.57292343]
 [0.4308809  0.5691191 ]
 [0.43089751 0.56910249]
 [0.42524941 0.57475059]
 [0.4318351  0.5681649 ]
 [0.43452608 0.56547392]
 [0.43479072 0.56520928]
 [0.43464078 0.56535922]
 [0.43781927 0.56218073]
 [0.43594655 0.56405345]
 [0.43682713 0.56317287]
 [0.43719319 0.56280681]
 [0.43920576 0.56079424]
 [0.43958564 0.56041436]
 [0.44330369 0.55669631]
 [0.43519453 0.56480547]
 [0.43503923 0.56496077]
 [0.42290866 0.57709134]
 [0.43178285 0.56821715]
 [0.41355606 0.58644394]
 [0.40927541 0.59072459]
 [0.42775034 0.57224966]
 [0.43043424 0.56956576]
 [0.43578083 0.56421917]
 [0.42833422 0.5716

In [36]:
# Data preprocessing
class DataPreprocessing(object): 
    def __init__(self, input_data_folder = 'C:/Users/Marius/Desktop/Daily data/',  output_data_folder='C:/Users/Marius/Desktop/Daily data/US_largest_stocks_Oxford/', 
                 output_data_and_kg_data_folder = 'C:/Users/Marius/Desktop/Daily data/US_largest_stocks_with_KG_data_Oxford/'): 
        self.input_data_folder = input_data_folder
        self.output_data_folder = output_data_folder
        self.output_data_and_kg_data_folder = output_data_and_kg_data_folder

    def prices_data_preprocessing(self, prices_file_name = 'all_largest_us_stocks_close_prices'):
        prices = pd.read_csv(self.input_data_folder + prices_file_name + '.csv', index_col = 0)
        prices.index = pd.to_datetime(prices.index)

        daily_pct_change = prices.pct_change(periods=1)

        for stock in prices.columns[:] : 
            prices_single_stock = prices[stock]
            daily_pct_change_single_stock = daily_pct_change[stock]
            daily_pct_change_single_stock.name = 'y'
            all_periods_single_stock = pd.DataFrame()
            for lag_period in [5, 10, 15, 21, 42, 63, 84, 105, 126, 147, 168, 189, 210, 231, 252] :
                pct_change_lag_period = prices_single_stock.pct_change(periods=lag_period)
                pct_change_lag_period.name = str(lag_period) + '_d'
                all_periods_single_stock = pd.concat([all_periods_single_stock, pd.DataFrame(pct_change_lag_period)], axis = 1)
            all_periods_single_stock_1d_lag_X = all_periods_single_stock.shift(1)
            all_periods_single_stock_1d_lag_X_y = pd.concat([all_periods_single_stock_1d_lag_X, pd.DataFrame(daily_pct_change_single_stock)], axis=1)
            all_periods_single_stock_1d_lag_X_y.to_csv(self.output_data_folder + stock + '.csv')


    def kg_and_tabular_data_preprocessing_single_stock(self, kg_columns_to_add = ['count'], kg_columns_to_add_names = 'count_1_or_2_nodes',
                                                        kg_data_file_name = 'number_of_stocks_1_or_2_nodes_away_with_corr_abv_0.8_since_2014-01-01'):
        kg_data = pd.read_csv(self.input_data_folder + kg_data_file_name + '.csv', index_col = 0)
        kg_data['date'] = pd.to_datetime(kg_data['date'], yearfirst = True)
        print(kg_data)

        for stock in os.listdir(self.output_data_folder)[:] : 
            stock_ticker = stock.split('.')[0]
            tabular_data = pd.read_csv(self.output_data_folder + stock, index_col = 0)
            tabular_data.index = pd.to_datetime(tabular_data.index, yearfirst = True)
            
            
            kg_single_stock_data = kg_data.where(kg_data['stock'] == stock_ticker, np.nan)
            kg_single_stock_data = kg_single_stock_data.dropna(how='all')
            kg_single_stock_data = kg_single_stock_data.reset_index().set_index('date')
            kg_single_stock_data = kg_single_stock_data[kg_columns_to_add]
            kg_single_stock_data.rename(columns = {kg_single_stock_data.columns[0] : kg_columns_to_add_names}, inplace=True)

            kg_and_tabular_data = pd.merge(tabular_data, kg_single_stock_data, how = 'left', left_index = True, right_index = True)
            kg_and_tabular_data[kg_columns_to_add_names] = kg_and_tabular_data[kg_columns_to_add_names].shift(1)

            kg_and_tabular_data.to_csv(self.output_data_and_kg_data_folder  + stock_ticker + '.csv')

    def kg_and_tabular_data_preprocessing_single_stock_updates(self, kg_columns_to_add = ['avg_dividends'], kg_columns_to_add_names = 'avg_dividends_1_or_2_nodes',    # avg_dividends,avg_splits
                                                        kg_data_file_name = 'all_avg_dividends_and_stock_splits_1_or_2_nodes_since_2014_01_01'):
        kg_data = pd.read_csv(self.input_data_folder + kg_data_file_name + '.csv', index_col = 0)
        kg_data['date'] = pd.to_datetime(kg_data['date'], yearfirst = True)
        print(kg_data)

        for stock in os.listdir(self.output_data_and_kg_data_folder)[:] : 
            stock_ticker = stock.split('.')[0]
            tabular_data = pd.read_csv(self.output_data_and_kg_data_folder + stock, index_col = 0)
            tabular_data.index = pd.to_datetime(tabular_data.index, yearfirst = True)
            
            
            kg_single_stock_data = kg_data.where(kg_data['stock'] == stock_ticker, np.nan)
            kg_single_stock_data = kg_single_stock_data.dropna(how='all')
            kg_single_stock_data = kg_single_stock_data.reset_index().set_index('date')
            kg_single_stock_data = kg_single_stock_data[kg_columns_to_add]
            kg_single_stock_data.rename(columns = {kg_single_stock_data.columns[0] : kg_columns_to_add_names}, inplace=True)

            kg_and_tabular_data = pd.merge(tabular_data, kg_single_stock_data, how = 'left', left_index = True, right_index = True)
            kg_and_tabular_data[kg_columns_to_add_names] = kg_and_tabular_data[kg_columns_to_add_names].shift(1)

            kg_and_tabular_data.to_csv(self.output_data_and_kg_data_folder  + stock_ticker + '.csv')  

    def kg_and_tabular_data_preprocessing_all_stock_aggregate_updates(self, kg_columns_to_add = ['avg_splits'], kg_columns_to_add_names = 'avg_splits_1_or_2_nodes',    # avg_dividends,avg_splits
                                                        kg_data_file_name = 'all_avg_dividends_and_stock_splits_1_or_2_nodes_since_2014_01_01'):
        kg_data = pd.read_csv(self.input_data_folder + kg_data_file_name + '.csv', index_col = 0)
        kg_data['date'] = pd.to_datetime(kg_data['date'], yearfirst = True)
        print(kg_data)

        for stock in os.listdir(self.output_data_and_kg_data_folder)[:] : 
            stock_ticker = stock.split('.')[0]
            tabular_data = pd.read_csv(self.output_data_and_kg_data_folder + stock, index_col = 0)
            tabular_data.index = pd.to_datetime(tabular_data.index, yearfirst = True)
            
            
            kg_single_stock_data = kg_data.copy()    #.where(kg_data['stock'] == stock_ticker, np.nan)
            kg_single_stock_data = kg_single_stock_data.dropna(how='all')
            kg_single_stock_data = kg_single_stock_data.reset_index().set_index('date')
            kg_single_stock_data = kg_single_stock_data[kg_columns_to_add]
            kg_single_stock_data.rename(columns = {kg_single_stock_data.columns[0] : kg_columns_to_add_names}, inplace=True)

            kg_and_tabular_data = pd.merge(tabular_data, kg_single_stock_data, how = 'left', left_index = True, right_index = True)
            kg_and_tabular_data[kg_columns_to_add_names] = kg_and_tabular_data[kg_columns_to_add_names].shift(1)

            kg_and_tabular_data.to_csv(self.output_data_and_kg_data_folder  + stock_ticker + '.csv')   

    def update_output_y_values_to_multi_day_returns(self, period_returns_days = 21) : 
        '''
        Use if returns period is one of the features X
        '''
        for stock in os.listdir(self.output_data_and_kg_data_folder) : 
            stock_ticker = stock.split('.')[0]
            tabular_data = pd.read_csv(self.output_data_and_kg_data_folder + stock, index_col = 0)
            tabular_data.index = pd.to_datetime(tabular_data.index, yearfirst = True)

            tabular_data['y'] = tabular_data[str(period_returns_days) + '_d'].shift(-1 * period_returns_days)
            tabular_data = pd.read_csv(self.output_data_and_kg_data_folder + stock)
            


if __name__ == "__main__":
    daily = DataPreprocessing()
    #daily.prices_data_preprocessing()
    
    #daily.kg_and_tabular_data_preprocessing_single_stock()
    #daily.kg_and_tabular_data_preprocessing_single_stock_updates()
    #daily.kg_and_tabular_data_preprocessing_all_stock_aggregate_updates()

    daily.update_output_y_values_to_multi_day_returns()



In [9]:
kg_data = pd.read_csv('C:/Users/Marius/Desktop/Daily data/US_largest_stocks_with_KG_data_Oxford/'+ 'BAC' + '.csv', index_col = 0)
#kg_data['date'] = pd.to_datetime(kg_data['date'], yearfirst = True)
print( kg_data.columns)

Index(['5_d', '10_d', '15_d', '21_d', '42_d', '63_d', '84_d', '105_d', '126_d',
       '147_d', '168_d', '189_d', '210_d', '231_d', '252_d', 'y',
       'count_1_or_2_nodes', 'count_1_nodes', 'avg_corr_1_nodes',
       'avg_corr_1_or_2_nodes', 'avg_dividends_1_or_2_nodes',
       'avg_splits_1_or_2_nodes'],
      dtype='object')


In [44]:
kg_data = pd.read_csv('C:/Users/Marius/Desktop/Daily data/'+ 'all_avg_dividends_and_stock_splits_1_or_2_nodes_since_2014_01_01' + '.csv', index_col = 0)
kg_data['date'] = pd.to_datetime(kg_data['date'], yearfirst = True)
print(kg_data)

         date  avg_dividends  avg_splits
0  2014-01-02       0.297590    0.277819
0  2014-01-03       0.297590    0.277819
0  2014-01-06       0.297590    0.277819
0  2014-01-07       0.306627    0.277819
0  2014-01-08       0.306627    0.277819
..        ...            ...         ...
0  2024-03-19       0.967831    0.530831
0  2024-03-20       0.967831    0.530831
0  2024-03-21       0.967831    0.530831
0  2024-03-22       0.967831    0.530831
0  2024-03-25       0.967831    0.530831

[2573 rows x 3 columns]


In [ ]:
# Example for train test split 

solar_numeric_X = solar_numeric_X_y.loc[:,:'Diadochite']
solar_numeric_y = solar_numeric_X_y['y']

X_solar_numeric_train, X_solar_numeric_test, y_solar_numeric_train, y_solar_numeric_test = train_test_split(solar_numeric_X, solar_numeric_y,test_size=0.2)

solar_scaler_supervised = StandardScaler()

X_solar_numeric_train_scaled = solar_scaler_supervised.fit_transform(X_solar_numeric_train)

X_solar_numeric_test_scaled = solar_scaler_supervised.transform(X_solar_numeric_test)

In [ ]:
# Random Forest Classifier

In [7]:
# ML project data preprocessing

full_data_single_stock = pd.read_csv('C:/Users/Marius/Desktop/Daily data/US_largest_stocks_with_KG_data_Oxford/'+ 'BAC' + '.csv', index_col = 0)
full_data_single_stock.index = pd.to_datetime(full_data_single_stock.index, yearfirst = True)

print(full_data_single_stock.iloc[:,-5:], full_data_single_stock.columns)




            count_1_nodes  avg_corr_1_nodes  avg_corr_1_or_2_nodes  \
2005-01-03            NaN               NaN                    NaN   
2005-01-04            NaN               NaN                    NaN   
2005-01-05            NaN               NaN                    NaN   
2005-01-06            NaN               NaN                    NaN   
2005-01-07            NaN               NaN                    NaN   
...                   ...               ...                    ...   
2024-03-19            3.0          0.842082               0.842095   
2024-03-20            3.0          0.842082               0.842095   
2024-03-21            3.0          0.842082               0.842095   
2024-03-22            3.0          0.842082               0.842095   
2024-03-25            3.0          0.842082               0.842095   

            avg_dividends_1_or_2_nodes  avg_splits_1_or_2_nodes  
2005-01-03                         NaN                      NaN  
2005-01-04                 

In [ ]:
# Random Forest Classifier
rfc_regularized = RandomForestClassifier(max_depth = 2, max_features = 3)
rfc_regularized.fit(X_solar_numeric_train_scaled, y_solar_numeric_train)
rfc_regularized_score = rfc_regularized.score(X_solar_numeric_test_scaled, y_solar_numeric_test)

print(f"Random Forrest Classifier Score = {rfc_regularized_score:.3f}")


In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

predictions_rfc_regularized = rfc_regularized.predict(X_solar_numeric_test_scaled)
cm = confusion_matrix(y_solar_numeric_test, predictions_rfc_regularized, labels=rfc_regularized.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
display_labels=rfc_regularized.classes_)
disp.plot()
plt.show()
print(cm)

In [ ]:
from sklearn.metrics import classification_report

print('Random Forest Classifier Regularized report table')
print(classification_report(y_solar_numeric_test, predictions_rfc_regularized))

In [ ]:
# Logistic regression

In [ ]:
from sklearn.linear_model import LogisticRegression

# Logistic regression
logModel = LogisticRegression(solver='liblinear')
logModel.fit(X_solar_numeric_train_scaled, y_solar_numeric_train)
logModel_score = logModel.score(X_solar_numeric_test_scaled, y_solar_numeric_test)
print(logModel_score)


In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

predictions_log = logModel.predict(X_solar_numeric_test_scaled)
cm = confusion_matrix(y_solar_numeric_test, predictions_log, labels=logModel.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
display_labels=logModel.classes_)
disp.plot()
plt.show()
print(cm)

In [ ]:
from sklearn.metrics import classification_report

print('Logistic regression classification report table')
print(classification_report(y_solar_numeric_test, predictions_log))